This is the basic brain model made by adding  hidden layers between the input and the output layer The model is created using pyTorch The model is made to run on  epochs for use now. 

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
train_df = pd.read_csv("C:\\Users\\Mokshda Sharma\\Desktop\\My Projects\\Biomimicry\\train.csv")
train_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [3]:
test_df = pd.read_csv("C:\\Users\\Mokshda Sharma\\Desktop\\My Projects\\Biomimicry\\test.csv")
test_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6
2,458996,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6
3,459000,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6
4,459001,Female,No,19,No,Marketing,NaN,Low,4.0,Cat_6


In [4]:
numerical_cols = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()

In [5]:
print(numerical_cols)
print(categorical_cols)

['ID', 'Age', 'Work_Experience', 'Family_Size']
['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1', 'Segmentation']


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [7]:
train_df['Work_Experience'].fillna(train_df['Work_Experience'].mean(), inplace=True)
train_df['Family_Size'].fillna(train_df['Family_Size'].mean(), inplace=True)

C:\Users\Mokshda Sharma\AppData\Local\Temp\ipykernel_3380\3161636716.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Work_Experience'].fillna(train_df['Work_Experience'].mean(), inplace=True)
C:\Users\Mokshda Sharma\AppData\Local\Temp\ipykernel_3380\3161636716.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we a

In [8]:
label_encoders = {}
label_encoder = LabelEncoder()
for col in categorical_cols:
    train_df[col] = label_encoder.fit_transform(train_df[col])

In [9]:
scaler = StandardScaler()
numeric_cols = ['Age', 'Work_Experience', 'Spending_Score', 'Family_Size']
train_df[numeric_cols] = scaler.fit_transform(train_df[numeric_cols])

In [10]:
X = train_df.drop(['ID', 'Segmentation'], axis=1)  
y = train_df['Segmentation']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)  
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)  
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [13]:
class CustomerSegmentationModel(nn.Module):
    def __init__(self, input_dim):
        super(CustomerSegmentationModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, 256)  
        self.layer2 = nn.Linear(256, 128)           
        self.layer3 = nn.Linear(128, 64) 
        self.layer4 = nn.Linear(64, 32) 
        self.layer5 = nn.Linear(32, 16)  
        self.layer6 = nn.Linear(16, 8)         
        self.output_layer = nn.Linear(8, 4)    
        self.relu = nn.ReLU()                  
    
    def forward(self, x, z=None):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.relu(self.layer4(x))
        x = self.relu(self.layer5(x))
        x = self.relu(self.layer6(x))
        x = self.output_layer(x)
        return x


In [14]:
model = CustomerSegmentationModel(input_dim=X_train_tensor.shape[1] )
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [15]:
# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    optimizer.zero_grad()  # Clear previous gradients
    
    # Forward pass
    outputs = model(X_train_tensor)  # Only pass X_train_tensor (no Z)
    
    # Compute loss
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [10/50], Loss: 1.3808
Epoch [20/50], Loss: 1.3123
Epoch [30/50], Loss: 1.2612
Epoch [40/50], Loss: 1.2115
Epoch [50/50], Loss: 1.1691


In [16]:
# Evaluation loop
model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient computation for evaluation
    # Forward pass for evaluation
    test_outputs = model(X_test_tensor)  # Only pass X_test_tensor (no Z)
    
    # Get predicted class
    _, predictions = torch.max(test_outputs, 1)
    
    # Compute accuracy
    accuracy = (predictions == y_test_tensor).float().mean()
    print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.4436
